![xframe](images/xframe.png)

<center> <h1>xframe is a dataframe for C++, based on xtensor and xtl</h1> </center>

In [1]:
#include <string>
#include <iostream>

#include "xtl/xbasic_fixed_string.hpp"

#include "xtensor/xio.hpp"
#include "xtensor/xrandom.hpp"
#include "xtensor/xmath.hpp"

#include "xframe/xvariable.hpp"
#include "xframe/xvariable_view.hpp"
#include "xframe/xvariable_masked_view.hpp"
#include "xframe/xio.hpp"

In [2]:
using fstring = xtl::xfixed_string<55>;

using data_type = xt::xoptional_assembly<xt::xarray<double>, xt::xarray<bool>>;
using coordinate_type = xf::xcoordinate<fstring, data_type::size_type>;

using variable_type = xf::xvariable<coordinate_type, data_type>;

## Variable with 2D Data 

In [3]:
data_type temperature_data = xt::abs(xt::random::randn<double>({6, 3})) * 10;
temperature_data(0, 0).has_value() = false;
temperature_data(2, 1).has_value() = false;

In [4]:
temperature_data

N/A,1.46382,4.6065
18.7138,1.63712,2.14253
2.98595,N/A,0.102154
10.5547,5.46841,11.7457
6.60682,10.4944,14.8596
6.25276,25.5912,8.29081


In [5]:
auto time_axis = xf::axis({"2018-01-01", "2018-01-02", "2018-01-03", "2018-01-04", "2018-01-05", "2018-01-06"});

In [6]:
auto temperature = variable_type(
    temperature_data,
    {
        {"date", time_axis},
        {"city", xf::axis({"London", "Paris", "Brussels"})}
    }
);

In [7]:
temperature

2018-01-01,N/A,1.46382,4.6065
2018-01-02,18.7138,1.63712,2.14253
2018-01-03,2.98595,N/A,0.102154
2018-01-04,10.5547,5.46841,11.7457
2018-01-05,6.60682,10.4944,14.8596
2018-01-06,6.25276,25.5912,8.29081


In [8]:
temperature.select({{"city", "London"}, {"date", "2018-01-04"}})

10.5547

In [9]:
temperature.locate("2018-01-03", "Brussels")

0.102154

In [10]:
temperature.locate("2018-01-03", "Paris")

N/A

In [11]:
data_type wind_factor_data = xt::abs(xt::random::randn<double>({6, 2}));
wind_factor_data(2, 0).has_value() = false;

In [12]:
auto wind_factor = variable_type(
    wind_factor_data, 
    {
        {"date", time_axis},
        {"city", xf::axis({"Paris", "Brussels"})}
    }
);

In [13]:
wind_factor

2018-01-01,0.539781,0.888707
2018-01-02,0.628956,1.01922
2018-01-03,N/A,0.482589
2018-01-04,2.10886,0.121306
2018-01-05,0.287389,0.371003
2018-01-06,1.05935,2.30144


In [14]:
variable_type perceived_temperature = temperature * wind_factor;

In [15]:
perceived_temperature

2018-01-01,0.79014,4.09383
2018-01-02,1.02968,2.18372
2018-01-03,N/A,0.0492986
2018-01-04,11.5321,1.42482
2018-01-05,3.01599,5.51297
2018-01-06,27.11,19.0808


## Variable with 3D data

In [16]:
data_type pressure_data = {{{ 1.,  2., 3. },
                            { 4.,  5., 6. },
                            { 7.,  8., 9. }},
                           {{ 1.3, 1.5, 1.},
                            { 2., 2.3, 2.4},
                            { 3.1, 3.8, 3.}},
                           {{ 8.5, 8.2, 8.6},
                            { 7.5, 8.6, 9.7},
                            { 4.5, 4.4, 4.3}}};

In [17]:
auto pressure = variable_type(
    pressure_data,
    {
        {"x", xf::axis(3)},
        {"y", xf::axis(3, 6, 1)},
        {"z", xf::axis(3)},
    }
);

In [18]:
pressure

0 1 2 0 3 1 2 3 4 4 5 6 5 7 8 9 1 3 1.3 1.5 1 4 2 2.3 2.4 5 3.1 3.8 3 2 3 8.5 8.2 8.6 4 7.5 8.6 9.7 5 4.5 4.4 4.3

In [19]:
pressure.select({{"x", 1}, {"y", 4}, {"z", 1}})

2.3

In [20]:
pressure.locate(0, 5, 2)

9

In [21]:
auto masked_pressure = xf::where(
    pressure, 
    not_equal(pressure.axis<int>("x"), 2) && pressure.axis<int>("z") < 2
);

In [22]:
masked_pressure

0 1 2 0 3 1 2 N/A 4 4 5 N/A 5 7 8 N/A 1 3 1.3 1.5 N/A 4 2 2.3 N/A 5 3.1 3.8 N/A 2 3 N/A N/A N/A 4 N/A N/A N/A 5 N/A N/A N/A